In [292]:
from github import Github, GithubException, Repository, ContentFile
import re

In [293]:
g = Github("dummy")

In [462]:
class ApiInstances:
    def getApiVersions(self, api_repo: Repository):
        try:
            app_contents = api_repo.get_contents("app")
            api_versions = []
            for app_content in app_contents:
                if re.match(version_regex, app_content.path) is not None:
                    api_version = app_content.path[4:]
                    api_versions.append(self.getApiVersionErrors(api_repo, api_version))
            return api_versions
        except Exception as ex:
            raise ex
            
    def getApiVersionErrors(self, api_repo: Repository, api_version: str):
        try:
            error_version = api_repo.get_contents("app/" + api_version + "/models/errors/mtdErrors.scala")
        except GithubException as ex:
            if ex.status == 404:
                error_version = api_repo.get_contents("app/" + api_version + "/models/errors/backendErrors.scala")
            else:
                raise ex
        return ErrorVersion(api_version, error_version)
        
            
    def __init__(self, api_repo: Repository):
        self.service_name = api_repo.name
        self.repo: Repository = api_repo
        self.api_versions = self.getApiVersions(api_repo)

In [553]:
class ErrorVersion:
    def __init__(self, api_version: str, raw_error_content: ContentFile):
        self.api_version = api_version
        self.error_content = self.getErrorContent(raw_error_content)
        self.has_paths = self.hasPaths(raw_error_content.repository)
        self.errors = self.getErrors(raw_error_content.repository)
        
    def hasPaths(self, repo: Repository):
        reg = "(?<=case class MtdError\().*?(?=\))"
        ans = re.search(reg, self.error_content)
        
        if ans == None:
            try:
                error_content = repo.get_contents("app/" + self.api_version + "/models/errors/MtdError.scala")
            except GithubException as ex:
                if ex.status == 404:
                    error_content = repo.get_contents("app/" + self.api_version + "/models/errors/Error.scala")
                    reg = "(?<=case class Error\().*?(?=\))"
                else:
                    raise ex
            
            ans = re.search(reg, self.getErrorContent(error_content))
            if ans == None:
                raise RuntimeError("a", "b")
        
        
        error_params = ans.group().split(",")
        for error_param in error_params:
            if "paths" in error_param:
                return True
        return False
    
    def getErrorContent(self, raw_error_content: ContentFile):
        boilerplate_regex = "\/\*\n(.*\*.*\n)*.*\*\/|.*\/\/.*\n|package.*\n|import.*\n"
        decoded_error_content = raw_error_content.decoded_content.decode("utf-8")
        error_content_without_comments = re.sub(boilerplate_regex, "", decoded_error_content)
        return error_content_without_comments
    
    def getErrors(self, repo: Repository):
        error_regex = "object *[A-Za-z]* *extends *[A-Za-z]*Error\(.*?\)"
        aa = repo.get_contents("app/" + self.api_version + "/models/errors")
        for file in aa:
            raw_error_strings = re.findall(error_regex, self.getErrorContent(file), re.DOTALL)
            errors = []
            for raw_error_string in raw_error_strings:
                errors.append(Error(raw_error_string.replace("\n", "")))
            if errors != []:
                return errors
        print(repo.name)
        raise RuntimeError("a", "b")

In [554]:
class Error:
    def __init__(self, raw_error_string: str):
        self.error_name = self.getErrorName(raw_error_string)
        self.error_code = ""
        self.error_message = ""
        
    def getErrorName(self, raw_error_string: str):
        error_name = re.search("(?<=object).*(?=.*extends.*Error\(.*?\))", raw_error_string)
        return error_name.group().replace(" ","")

In [555]:
api_instances = []
        
mtd_repos = g.get_organization("hmrc").get_team(1996975).get_repos()
version_regex = "app/v[\d]+"

for repo in mtd_repos:
    if repo.name.endswith("api"):
        api_instances.append(ApiInstances(repo))